## Для обучения второй модели сделаем эмбеддинги из текстов наших постов с помощью предобученных моделей Hugging Face.

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

import re

import sys
import gc
import os

from dotenv import load_dotenv

from tqdm import tqdm

from transformers import AutoTokenizer
from transformers import RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
from transformers import DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding    

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 7.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.6-cp310-cp310-linux_x86_64.whl size=168900 sha256=879b66e8afb0416d2413272b052eaf154e9c2de99b10194388ec7c1bb23f9603
  Stored in directory: /root/.cache/pip/wheels/a2/65/83/78e6f42d3b8e22115e894576b71799d96ab5a790b8f7bcfa85
Successfully built psycopg2
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sqlalchemy import create_engine, text


engine = create_engine("postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml")
query = 'SELECT * FROM public.post_text_df'

posts_info = pd.read_sql_query(sql=text(query), con=engine.connect())

In [5]:
# Класс создаёт набор данных из токенизированных текстов.
# Тексты токенизируется сразу же токенизатором tokenizer, при создании экземпляра класса.
class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        super().__init__()

        self.texts = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {'input_ids': self.texts['input_ids'][idx], 'attention_mask': self.texts['attention_mask'][idx]}

    def __len__(self):
        return len(self.texts['input_ids'])

In [6]:
# Функция для загрузки модели roberta и distilbert из huggingface
def get_model(model_name):
    
    assert model_name in ['roberta', 'distilbert']

    checkpoint_names = {
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }

    model_classes = {
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [7]:
# Подсчёт моделью эмбеддингов токенизированных текстов.
# Эмбеддинги берутся с первого слоя last_hidden_state.
@torch.inference_mode()
def get_embeddings_labels(model, loader, device):
    model.eval()
    
    total_embeddings = []
    
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [8]:
tokenizer, model = get_model('distilbert')

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
dataset = PostDataset(posts_info['text'].values.tolist(), tokenizer)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)
batch_data = next(iter(loader))

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
out_batch = model(**batch_data)
print(out_batch['last_hidden_state'][:, 0, :].shape)

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name())

model = model.to(device)

cuda:0
Tesla T4


In [12]:
embeddings = get_embeddings_labels(model, loader, device).numpy()

100%|██████████| 220/220 [02:01<00:00,  1.82it/s]


### Пытаемся снизить размерность и кластеризовать тексты с помощью PCA и KNN также как для первой модели.

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=70)
pca_decomp = pca.fit_transform(posts_info_PCA)
pca_decomp

In [14]:
from sklearn.cluster import KMeans

n_clusters = 15

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pca_decomp)

posts_info['TextCluster'] = kmeans.labels_

dists_columns = [f'DistanceToCluster_{i}' for i in range(n_clusters)]

dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,DistanceToCluster_0,DistanceToCluster_1,DistanceToCluster_2,DistanceToCluster_3,DistanceToCluster_4,DistanceToCluster_5,DistanceToCluster_6,DistanceToCluster_7,DistanceToCluster_8,DistanceToCluster_9,DistanceToCluster_10,DistanceToCluster_11,DistanceToCluster_12,DistanceToCluster_13,DistanceToCluster_14
0,2.375404,3.665928,3.373343,3.443877,3.622818,3.386910,1.812277,3.004539,2.834371,2.249783,3.473739,1.900510,3.408670,3.412318,3.462497
1,2.329184,3.468094,3.325495,2.982847,3.358610,3.371622,2.184574,2.850791,2.552490,2.243558,3.248824,1.426576,3.326208,3.216277,3.136622
2,2.398518,3.451669,3.265284,2.972764,3.367133,3.501859,1.858817,3.039336,2.885038,3.061794,3.397064,1.684046,3.358547,3.287411,3.133440
3,2.824766,3.153519,3.513383,3.718793,3.796568,3.748973,2.461983,3.278253,3.374481,3.411346,4.065657,2.435080,3.739384,3.696885,3.790943
4,2.040599,3.164101,3.033904,2.641305,3.048451,2.803077,1.463832,2.642233,2.137744,2.950977,3.241409,2.114441,2.807246,2.842687,2.775343


In [15]:
posts_info = pd.concat((posts_info, dists_df), axis=1)

### Загрузим признаки для второй модели на сервер 

In [22]:
posts_info.head(2)
posts_info.to_sql("ryabgri_post_inf_features_emb_3", con=engine, if_exists='replace')

,post_id,topic,DistanceToCluster_0,DistanceToCluster_1,DistanceToCluster_2,DistanceToCluster_3,DistanceToCluster_4,DistanceToCluster_5,DistanceToCluster_6,DistanceToCluster_7,DistanceToCluster_8,DistanceToCluster_9,DistanceToCluster_10,DistanceToCluster_11,DistanceToCluster_12,DistanceToCluster_13,DistanceToCluster_14,text
0,1,business,2.375404,3.665928,3.373343,3.443877,3.622818,3.386910,1.812277,3.004539,2.834371,2.249783,3.473739,1.900510,3.408670,3.412318,3.462497,UK economy facing major risks\n\nThe UK manufa...
1,2,business,2.329184,3.468094,3.325495,2.982847,3.358610,3.371622,2.184574,2.850791,2.552490,2.243558,3.248824,1.426576,3.326208,3.216277,3.136622,Aids and climate top Davos agenda\n\nClimate c...
